In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_digits

digits = load_digits()

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3)

In [4]:
lr = LogisticRegression(max_iter=3000) #lr is called a classifier which is an ML model
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.975925925925926

In [5]:
svm = SVC()
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.9925925925925926

In [6]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.975925925925926

In [7]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

0.5739756164634706

### Conclusion


Every time you run the split method (e.g., `X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3)`), the model's performance is affected, resulting in slightly different outcomes. This means that a model that was previously the best performer might rank second or vice versa. This variability occurs because the `train_test_split` function randomly generates a different training and testing dataset on each execution, unless a random seed is specifieand.

### Let's try with K-Fold cross validation

**K-Fold Cross-Validation Method:**  
K-Fold cross-validation is a technique that provides train/test indices to split a dataset into train and test sets. It divides the dataset into *k* consecutive folds (without shuffling by default). Each fold is used once as a validation set, while the remaining *k - 1* folds are combined to form the training set.  

For more details, refer to the official documentation: [scikit-learn KFold](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.KFold.html).  

In [8]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [9]:
for train_index, text_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, text_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


In [10]:
def get_score(model,X_train,X_test,y_train, y_test ):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [11]:
get_score(SVC(),X_train,X_test,y_train, y_test)

0.9925925925925926

In [12]:
get_score(DecisionTreeRegressor(),X_train,X_test,y_train, y_test)

0.561829681914016

##### StratifiedKFold

In [13]:
from sklearn.model_selection import StratifiedKFold #categorize in an uniform way
folds =  StratifiedKFold(n_splits=3)

In [14]:
score_lr = []
score_svm =[]
score_rf = []
score_dt = []

#This `for` loop is intended solely to demonstrate the process.
for train_index, test_index in kf.split(digits.data):
    X_train,X_test,y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                        digits.target[train_index], digits.target[test_index]
    score_lr.append(get_score(LogisticRegression(max_iter=3000),X_train,X_test,y_train, y_test))
    score_rf.append(get_score(RandomForestClassifier(n_estimators=40),X_train,X_test,y_train, y_test))
    score_svm.append(get_score(SVC(),X_train,X_test,y_train, y_test))
    score_dt.append(get_score(DecisionTreeRegressor(),X_train,X_test,y_train, y_test))

In [15]:
score_lr

[0.9282136894824707, 0.9415692821368948, 0.9165275459098498]

In [16]:
score_rf

[0.9248747913188647, 0.9465776293823038, 0.9215358931552587]

In [17]:
score_svm

[0.9666110183639399, 0.9816360601001669, 0.9549248747913188]

In [18]:
score_dt

[0.5747334621481963, 0.5778952852943138, 0.5697220111514034]

In [19]:
avg_score_lr = np.mean(score_lr)
avg_score_lr

0.9287701725097385

In [20]:
avg_score_rf = np.mean(score_rf)
avg_score_rf

0.9309961046188091

In [21]:
avg_score_svm = np.mean(score_svm)
avg_score_svm

0.9677239844184752

In [22]:
avg_score_dt = np.mean(score_dt)
avg_score_dt

0.5741169195313045

In [23]:
best_model = max([avg_score_lr, avg_score_rf, avg_score_svm, avg_score_dt])
best_model

0.9677239844184752

##### The best-performing model is `avg_score_svm`, achieving an accuracy of 96.77%

##### We can streamline and accelerate these steps by using the `sklearn` function `cross_val_score`.

In [24]:
from sklearn.model_selection import cross_val_score

##### Instead of using a `for` loop, we can simply use:

In [25]:
cross_val_score(LogisticRegression(max_iter=30000),digits.data, digits.target, cv=3)

array([0.92487479, 0.93989983, 0.92320534])

In [26]:
cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target, cv=3)

array([0.91986644, 0.93989983, 0.92654424])

In [27]:
cross_val_score(SVC(),digits.data, digits.target, cv=3)

array([0.96494157, 0.97996661, 0.96494157])

In [28]:
cross_val_score(DecisionTreeRegressor(),digits.data, digits.target, cv=3)

array([0.58600947, 0.62556541, 0.57630056])

In [30]:
##Let's calculate the average 
print('LogisticRegression',np.mean(cross_val_score(LogisticRegression(max_iter=30000),digits.data, digits.target, cv=3)))
print('RandomForestClassifier',np.mean(cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target, cv=3)))
print('SVC',np.mean(cross_val_score(SVC(),digits.data, digits.target, cv=3)))
print('DecisionTreeRegressor', np.mean(cross_val_score(DecisionTreeRegressor(),digits.data, digits.target, cv=3)))

LogisticRegression 0.9293266555370061
RandomForestClassifier 0.9298831385642737
SVC 0.9699499165275459
DecisionTreeRegressor 0.5658768875205001


##### Now we can make the same conclusion The best-performing model is `avg_score_svm`, achieving an accuracy of 96.99%

This project is inspired by the following YouTube video: [How to Evaluate Models using Cross-Validation](https://www.youtube.com/watch?v=gJo0uNL-5Qw).